In [1]:
import os
import random
import torch
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import mixed_precision as mp
from ultralytics import YOLO
from roboflow import Roboflow
from PIL import Image
from datetime import datetime
from dotenv import load_dotenv
from shutil import copy2

In [2]:
# use gpu if available
physical_devices = tf.config.list_physical_devices('GPU')
print("Physical devices:", physical_devices)
if physical_devices:
    print("GPU available, enabling mixed precision training")
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
    policy = mp.Policy('mixed_float16')
    mp.set_global_policy(policy)
else:
    print("GPU not available, using CPU")

print("PyTorch CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

load_dotenv()

Physical devices: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
GPU available, enabling mixed precision training
INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3060, compute capability 8.6
PyTorch CUDA available: True
NVIDIA GeForce RTX 3060


True

In [3]:
def download_data(Roboflow, workspace_name, project_name, version, folderpath='datasets/player_tracking'):
    rf = Roboflow(api_key=os.getenv("ROBOFLOW_API_KEY"))
    project1 = rf.workspace(workspace_name).project(project_name)
    version = project1.version(version)
    dataset = version.download("yolov11", location=folderpath)
    return dataset

def train_model(YOLO, tf, folderpath='datasets/player_tracking'):
    model = YOLO('yolo11n.pt')
    print(model)
    model.info()
    train_results = model.train(
        data=f"{folderpath}/data.yaml",
        epochs=150,
        imgsz=640,
        batch=16,
        device=0 if tf.config.list_physical_devices('GPU') else 'cpu',
        workers=4,
        optimizer='Adam',
        lr0=0.01,
        lrf=0.01,
        patience=20,
        project="runs/detect/train",
        name="yolov11-player-fixed",
        save=True,
        save_period=10,
        exist_ok=True,
        verbose=True,
        plots=True,
        val=True,
    )
    return model, train_results

def evaluate_model(model):
    metrics = model.val()
    return metrics

def run_inference(model, image_path):
    results = model(image_path)
    results[0].show()
    return results

def export_model(model):
    path = model.export(format="onnx")
    return path

In [ ]:
# SKIP IF TRAINING/DATASETS/PLAYER_TRACKING ALREADY EXISTS

def download_roboflow_datasets(rf, datasets, base_folder):
    download_paths = []
    for ds in datasets:
        print(f"Downloading {ds['workspace']}/{ds['project']} version {ds['version']}...")
        workspace = rf.workspace(ds['workspace'])
        project = workspace.project(ds['project'])
        version = project.version(ds['version'])
        out_path = os.path.join(base_folder, f"{ds['project']}_v{ds['version']}")
        dataset = version.download('yolov11', location=out_path)
        print(f"Downloaded to: {dataset.location}")
        download_paths.append(dataset.location)
    return download_paths

rf = Roboflow(api_key=os.getenv("ROBOFLOW_API_KEY"))
dataset_folderpath = "datasets/player_tracking"
merge_datasets = [
    {"workspace": "volley-pjhnb", "project": "volleyball-q7cun", "version": 3},
    {"workspace": "volleyball-9ape8", "project": "volleyball-g4zfi", "version": 2},
    {"workspace": "david-paulinus-54fnl", "project": "tracking-volleyball-players", "version": 6},
]
download_paths = download_roboflow_datasets(rf, merge_datasets, dataset_folderpath)

In [ ]:
dataset_folderpath = "datasets/fixed_player_tracking"
# consolidated the above 3 datasets into one
# https://app.roboflow.com/daniel-88d0p/volleyball-players-wgaid/1
fixed_dataset = {"workspace": "daniel-88d0p", "project": "volleyball-players-wgaid", "version": 1}
dataset = download_data(Roboflow, workspace_name=fixed_dataset["workspace"], project_name=fixed_dataset["project"], version=fixed_dataset["version"], folderpath=dataset_folderpath)

loading Roboflow workspace...
loading Roboflow project...
Exporting format yolov11 in progress : 85.0%
Version export complete for yolov11 format



Extracting Dataset Version Zip to datasets/fixed_player_tracking in yolov11:: 100%|██████████| 7648/7648 [00:05<00:00, 1505.49it/s]


In [7]:
model, train_results = train_model(YOLO, tf, folderpath='datasets/fixed_player_tracking')

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C3k2(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_

In [6]:
metrics = evaluate_model(model)
# results = run_inference(model, "path/to/image.jpg")
onnx_path = export_model(model)

Ultralytics 8.3.233  Python-3.10.11 torch-2.1.0+cu118 CUDA:0 (NVIDIA GeForce RTX 3060, 12288MiB)
val: Fast image access  (ping: 0.10.0 ms, read: 114.6235.0 MB/s, size: 55.1 KB)
val: Scanning D:\Coding\tobio\training\datasets\player_tracking\players\valid\labels.cache... 738 images, 7 backgrounds, 333 corrupt: 100% ━━━━━━━━━━━━ 738/738  0.0s
val: D:\Coding\tobio\training\datasets\player_tracking\players\valid\images\1012_jpeg.rf.29fdb009012a109e9ddbfc78bfd10e9d.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: D:\Coding\tobio\training\datasets\player_tracking\players\valid\images\1044_jpeg.rf.6403c87c410ffa2e07a2ceebfb372dbb.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class count 1. Possible class labels are 0-0
val: D:\Coding\tobio\training\datasets\player_tracking\players\valid\images\1045_jpeg.rf.42623a0887d02f569e9ba41e0e568b8a.jpg: ignoring corrupt image/label: Label class 1 exceeds dataset class 

ImportError: cannot import name 'builder' from 'google.protobuf.internal' (d:\Coding\tobio\.venv\lib\site-packages\google\protobuf\internal\__init__.py)